In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re 
from scipy import stats

import seaborn as sns 

from selenium import webdriver

In [2]:
#Key 

#ulta_1 = moisturizes 
#ulta_2 = sunscreen 
#etc

In [3]:
### Get Chrome Driver Path 
path = r"C:\Users\avach\Downloads\clone\info2950\chromedriver.exe" 
driver = webdriver.Chrome(path) 

### Open Ulta Moisturizer Site
driver.get('https://www.ulta.com/skin-care-moisturizers?N=2796Z1z13p3l')

### Find first product and click to open 
prod = driver.find_element_by_xpath('//*[@id="xlsImpprod13641053"]/div[2]') 
prod.click()





In [17]:
ulta_1 = pd.DataFrame(columns = ['brand','name','price','rating','ingredients'])
ulta_1.head()

,brand,name,price,rating,ingredients


In [26]:
### Get brand
brand = driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/h1/div[1]').text
print(brand)

### Get Product Name 
name = driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/h1/div[2]').text
print(name) 
 

### Get Rating Number
star = driver.find_element_by_xpath('//*[@id="pr-review-snapshot"]/header/section/div/div[1]/div/div[1]/div').text
print(star) 

### Once on product page, toggle ingredients and get text
toggle = driver.find_element_by_xpath('//*[@id="productDescription"]/div[3]/div[2]')
toggle.click() 

ingredient = driver.find_element_by_xpath('//*[@id="productDescription"]/div[3]/div[2]/div[2]/div/div/div').text
print(ingredient) 

### Get Price 
price = driver.find_element_by_xpath('//*[@id="js-mobileBody"]/div/div/div/div/div/div/section[1]/div[2]/div/div[3]').text
print(price)

It Cosmetics
Confidence In A Cream Anti-Aging Moisturizer
4.6
Aqua (Water, Eau), Butylene Glycol, Cyclopentasiloxane, Glycerin, Butylene Glycol Dicaprylate / Dicaprate, Ethylhexyl Isononanoate, Hydrogenated Polyisobutene, Caprylic / Capric / Myristic / Stearic Triglyceride, Glyceryl Stearate, PEG - 100 Stearate, Polyglyceryl - 3 Methylglucose Distearate, Stearyl Behenate, Dimethicone, Pentaerythrityl Tetraethylhexanoate, Biosaccharide Gum - 1, Dimethicone / Vinyl Dimethicone Crosspolymer, Cetearyl Alcohol, Palmitic Acid, Stearic Acid, Polyethylene, PEG / PPG - 20 / 15 Dimethicone, Phenyl Methicone, Portulaca Oleracea Extract, Cocos Nucifera (Coconut) Oil, Aloe Barbadensis Leaf Extract, Butyrospermum Parkii (Shea) Butter, Hydroxyethyl Acrylate / Sodium Acryloyldimethyl Taurate Copolymer, Sodium Hyaluronate, Hydrogenated Lecithin, Polyquaternium - 51, Niacinamide, Hydrolyzed Collagen, Camellia Sinensis Leaf Extract, Tocopheryl Acetate, Carbomer, PEG - 150 Distearate, Ceramide Ap, Glycosp

In [1]:
def scrollDown(driver, n_scroll):
    body = driver.find_element_by_tag_name("body")
    while n_scroll >= 0:
        body.send_keys(Keys.PAGE_DOWN)
        n_scroll -= 1
    return driver